# Farmer Problem - Planting crops 

In [3]:
from pulp import *

In [4]:
# To show nice looking table of solution
from IPython.display import HTML, display

def display_table(data):
    html = "<table>"
    for row in data:
        html += "<tr>"
        for field in row:
            html += "<td><h4>%s</h4><td>"%(field)
        html += "</tr>"
    html += "</table>"
    display(HTML(html))

## Pick one of the yield scenarios and model will find optimal planting, buying and selling

In [5]:
SCENARIO = "AVERAGE" #AVERAGE, GOOD , BAD

# Business Problem as Table 

### This is base case - will not update if you are updating the numbers 

### Total acerage = 500

|Field|Wheat|Corn|Beans|
|-----|-------|------|----------|
|Yield T/acre|2.5|3|20|
|Planting cost $/acre|150|230|260|
|Selling price|170|150|36 (<6000) 10 (>6000)|
|Purchase price|238|210|NA|
|Farm Needs|200|240|NA|


### Yields vary based on weather

Prob|Yield|Wheat|Corn|Beans|
----|-----|-------|------|----------|
0.333|Bad |2|2.4|16|
0.333|Average|2.5|3|20|
0.333|Good|3|3.6|24|

## Start defining data needed for problem

### Define Items and Farm Acerage

In [6]:
total_acres = 500
items = ["Wheat", "Corn","Beans"]
purchasable_items = ["Wheat", "Corn"]                         

### Define yields and pick one yield based on scenario we are analyzing

In [7]:
yields_by_scenario = {"BAD": {"Wheat":2,"Corn":2.4,"Beans":16}, 
          "AVERAGE" : {"Wheat":2.5,"Corn":3,"Beans":20},
          "GOOD": {"Wheat":3,"Corn":3.6,"Beans":24}}
yields = yields_by_scenario[SCENARIO]


### Define Planting Costs and Market purchase prices  

In [8]:
planting_costs = {"Wheat":150,"Corn":230,"Beans":260}  
purchase_prices = {"Wheat":238,"Corn":210,"Beans":99999}

### Define consumption rules

In [9]:
consumption_feed={"Wheat":200,"Corn":240,"Beans":0}          

### Define selling prices - note the price difference for beans excess of 6000

In [10]:
selling_prices = {"Wheat":170,"Corn":150,"Beans":36}
limit_on_beans_regular_price = 6000
selling_price_excess_beans = 10 #33 and 35.25 are inflection points to stop producing corn and wheat and buy them

# Define Model

## Maximize profit (Revenue of selling - planting costs - buying costs)

In [11]:
M = LpProblem("Farmer", LpMaximize)

# Define Variables

In [12]:
var_acres_planted = LpVariable.dicts("plant", items,lowBound=0,cat='Continuous')
var_tons_purchased = LpVariable.dicts("purchase", purchasable_items,lowBound=0,cat='Continuous') #no purchase for beans
var_tons_sold = LpVariable.dicts("sell", items,lowBound=0,cat='Continuous')
var_excess_beans_sold = LpVariable('extra_beans', lowBound=0,cat='Continuous')


# Define Objective

In [13]:
M += lpSum( 
      [      selling_prices[item]       * var_tons_sold[item]        for item in items ]
    + [      selling_price_excess_beans * var_excess_beans_sold]     #excess beans
    + [ -1 * planting_costs[item]       * var_acres_planted[item]    for item in items ]  
    + [ -1 * purchase_prices[item]      * var_tons_purchased[item]   for item in purchasable_items ]    
)



# Constraints

In [14]:
M += lpSum([var_acres_planted[item] for item in items]) <= total_acres

for item in purchasable_items:
    M += lpSum([yields[item] * var_acres_planted[item]] + [var_tons_purchased[item]] + [-1 * var_tons_sold[item]]) == consumption_feed[item]
    
for item in list(set(items)-set(purchasable_items)): #beans - constraint needs one more variable var_excess_beans_sold
    M += lpSum([yields[item] * var_acres_planted[item]] + [-1 * var_tons_sold[item]] + [-1 * var_excess_beans_sold]) == consumption_feed[item]    
    M += lpSum([var_tons_sold[item]]) <= limit_on_beans_regular_price
    
                        
     

## Be Careful activating these constraints. 

In [15]:
#Be careful - these are meant to set acres for crops and evaluate profit
# The model no longer has freedom to make decisions
"""
M += lpSum([var_acres_planted["Wheat"]]) == 183.33333
M += lpSum([var_acres_planted["Corn"]]) == 66.66666
M += lpSum([var_acres_planted["Beans"]]) == 250
"""

'\nM += lpSum([var_acres_planted["Wheat"]]) == 183.33333\nM += lpSum([var_acres_planted["Corn"]]) == 66.66666\nM += lpSum([var_acres_planted["Beans"]]) == 250\n'

In [16]:
M.solve()
M.writeLP("Farmer_OneScenario_Model.lp")

[extra_beans,
 plant_Beans,
 plant_Corn,
 plant_Wheat,
 purchase_Corn,
 purchase_Wheat,
 sell_Beans,
 sell_Corn,
 sell_Wheat]

In [17]:
print("Status = %s" % LpStatus[M.status])
#print("%s = %f" % (order.name, order.varValue))
print("Profit = %f" % (M.objective.value()))

Status = Optimal
Profit = 118600.000000


In [18]:
t = []
Total_Planted = 0
Total_Revenue = 0
Total_Profit = 0

t.append(["Item","Planted","Produced", "Purchased", "Sold","Revenue","Plant Cost","Purchase Cost","Profit"])    
for item in items:
    purchased = 0.0
    if item in purchasable_items:
        purchased = var_tons_purchased[item].varValue
    
    planted = var_acres_planted[item].varValue
    planting_cost = planted* planting_costs[item]
    produced = yields[item] * var_acres_planted[item].varValue
    sold = var_tons_sold[item].varValue
    revenue = selling_prices[item]*sold
    if item=="Beans":
        sold += var_excess_beans_sold.varValue
        revenue += selling_price_excess_beans*var_excess_beans_sold.varValue
    purchase_cost = purchase_prices[item]*purchased
    profit = revenue - planting_cost - purchase_cost
    
    
    t.append([item, planted, produced, purchased, sold, revenue, planting_cost, purchase_cost, profit])
    Total_Planted += planted
    Total_Revenue += revenue
    Total_Profit += profit
    
t.append(["","","", "", "","","","",""])    

t.append(["Total","-",Total_Planted, "", "",Total_Revenue,"","",Total_Profit])    

    
display_table(t)

## To be added - Sensitivity Analysis on limits, buying and selling prices

In [19]:
#o = [{'name':name,'shadow price':c.pi, 'slack': c.slack} for name, c in M.constraints.items()]

In [20]:
savings_avoid_purchase = {item: yields[item]*purchase_prices[item]- planting_costs[item] for item in purchasable_items}
profit_acre = {item: yields[item]*selling_prices[item]- planting_costs[item] for item in items}
profit_from_excess_beans_per_acre = yields["Beans"]*selling_price_excess_beans- planting_costs["Beans"] 


In [21]:
# What if beans sales price decreases
# What if beans excess sales price increases
#Can you compute following variables to see if you can guess the optimal solution
#Hint: Sort the numbers below and try greedy approach
print("Savings from producing vs purchasing per acre:")
print(savings_avoid_purchase)
print("Profit per acre:")
print(profit_acre)
print(f" profit_from_excess_beans_per_acre {profit_from_excess_beans_per_acre}")

Savings from producing vs purchasing per acre:
{'Wheat': 445.0, 'Corn': 400}
Profit per acre:
{'Wheat': 275.0, 'Corn': 220, 'Beans': 460}
 profit_from_excess_beans_per_acre -60
